In [ ]:
! nvidia-smi

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sys
from nltk.tokenize import TweetTokenizer
from emoji import demojize

import random
import math

from tqdm.notebook import tqdm
import string

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold

import tensorflow as tf
from tensorflow.keras import backend as K

import transformers
from transformers import TFAutoModel, AutoTokenizer, TFAutoModelForSequenceClassification

# from fairseq.data.encoders.fastbpe import fastBPE
# from fairseq.data import Dictionary
# import argparse

# Load rdrsegmenter from VnCoreNLP
# from vncorenlp import VnCoreNLP

print('Using Tensorflow version:', tf.__version__)
print('Using Transformers version:', transformers.__version__)

# import warnings
# warnings.filterwarnings('ignore')
tqdm.pandas()

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
seed = 1512
def seed_all(seed=1512):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
seed_all(seed)

In [ ]:
data_dir = '../data/tokenize_data'
for fn in os.listdir(data_dir):
    print(os.path.join(data_dir, fn))

In [ ]:
train_df = pd.read_csv("../data/tokenize_data/train.csv")
test_df = pd.read_csv("../data/tokenize_data/test.csv")
print(len(train_df), len(test_df))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# label_map = {"UNINFORMATIVE": 0, "INFORMATIVE": 1}


# def get_labels(file_in, gold_indices=None):
#     """
#     Read the labels from file
#     :param file_in: path to the file
#     :param gold_indices: set of gold indices
#     :return: list of labels
#     """
#     labels = []
#     count = 0
#     with open(file_in) as reader:
#         for line in reader:
#             line = line.strip()
#             if line == "Id\tText\tLabel":
#                 continue
#             count += 1
#             if gold_indices is not None and count not in gold_indices:
#                 continue

#             label = line.split()[-1].upper()
#             if len(label) == 0:
#                 continue

#             if label in label_map:
#                 labels.append(label_map[label])
#             else:
#                 print("Error occurs at line {}. "
#                       "{} is not a label (only support UNINFORMATIVE and INFORMATIVE). "
#                       "Process terminated.".format(count + 1, label))
#                 sys.exit()


#     return labels


# def calculate_scores(pred_labels, true_labels, pos_label=label_map["INFORMATIVE"]):
#     """
#     Calculate the precision, recall, f1 and accuracy scores for the predictions
#     :param pred_labels: prediction labels
#     :param true_labels: ground truth labels
#     :param pos_label: INFORMATIVE label
#     :return: precision, recall, f1 and accuracy scores
#     """
#     assert len(pred_labels) == len(true_labels)

#     tp = 0  # true positive
#     fn = 0  # false negative
#     fp = 0  # false positive
#     n_correct = 0
#     for i in range(len(pred_labels)):
#         if true_labels[i] == pred_labels[i]:
#             n_correct += 1
#             if pred_labels[i] == pos_label:
#                 tp += 1
#         else:
#             if pred_labels[i] == pos_label:
#                 fp += 1
#             else:
#                 fn += 1
#     # Precision score
#     precision = 0.0
#     if tp + fp > 0:
#         precision = tp * 1.0 / (tp + fp)

#     # Recall score
#     recall = 0.0
#     if tp + fn > 0:
#         recall = tp * 1.0 / (tp + fn)

#     # F1 score
#     f1 = 0.0
#     if recall + precision > 0:
#         f1 = 2 * precision * recall / (precision + recall)

#     # Accuracy score
#     accuracy = 0.0
#     if len(true_labels) > 0:
#         accuracy = n_correct * 1.0 / len(true_labels)
#     return precision, recall, f1, accuracy


# def evaluate(pred_label_file, true_label_file, gold_indices_file=None):
#     gold_indices = get_gold_indices(gold_indices_file)
#     pred_labels = get_labels(pred_label_file, gold_indices)
#     true_labels = get_labels(true_label_file)
#     return calculate_scores(pred_labels, true_labels)


# def get_gold_indices(gold_indices_file):
#     if gold_indices_file is None:
#         return None

#     gold_indices = set()
#     with open(gold_indices_file, "r") as reader:
#         for line in reader:
#             line = line.strip()
#             if len(line) > 0:
#                 gold_indices.add(int(line))
#     return gold_indices


# def score(input_dir, output_dir):
#     # unzipped submission data is always in the 'res' subdirectory
#     submission_dir = os.path.join(input_dir, 'res')
#     submission_file = []
#     for el in os.listdir(submission_dir):
#         if el.startswith('predictions'):
#             submission_file.append(el)
#     if not len(submission_file) == 1:
#         print(
#             "Warning: the submission folder should only contain 1 file (i.e., 'predictions.txt'). Process terminated.")
#         sys.exit()
#     submission_file_name = submission_file[0]
#     submission_path = os.path.join(submission_dir, submission_file_name)
#     ground_truth_file = os.path.join(input_dir, 'ref', 'groundtruth_data.txt')
#     gold_indices_file = os.path.join(input_dir, 'ref', 'gold_indices.txt')
#     precision, recall, f1, accuracy = evaluate(submission_path, ground_truth_file, gold_indices_file)
#     with open(os.path.join(output_dir, 'scores.txt'), 'w') as output_file:
#         output_file.write("F1-score:{}\nPrecision:{}\nRecall:{}\nAccuracy:{}\n".format(f1, precision, recall, accuracy))


# def main():
#     [_, input_dir, output_dir] = sys.argv
#     score(input_dir, output_dir)


# # if __name__ == "__main__":
# #     # main()
# #     [_, pred_label_file, true_label_file] = sys.argv
# #     precision, recall, f1, accuracy = evaluate(pred_label_file, true_label_file)
# #     print("F1-score: {}\nPrecision: {}\nRecall: {}\nAccuracy: {}\n".format(f1, precision, recall, accuracy))

In [ ]:
roberta = 'vinai/phobert-base' 
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta)
roberta_model = TFAutoModel.from_pretrained(roberta)

In [ ]:
train_len_word = [len(text.split()) for text in train_df.post_message]
test_len_word = [len(text.split()) for text in test_df.post_message]
test_len_char = [len(text) for text in train_df.post_message]
test_len_char = [len(text) for text in test_df.post_message]

In [ ]:
def length_plot(lengths):
    plt.figure(figsize=(12,7))
    textstr = f' Mean: {np.mean(lengths):.2f} \u00B1 {np.std(lengths):.2f} \n Max: {np.max(lengths)}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

    plt.text(0, 0, textstr, fontsize=14,
            verticalalignment='top', bbox=props)
    sns.countplot(lengths)    

In [ ]:
# length_plot(train_len_word)

In [ ]:
# length_plot(test_len_word)

In [ ]:
MAX_LEN = 184
BATCH_SIZE = 32

In [ ]:
def regular_encode(texts, maxlen=MAX_LEN):

    roberta_enc_di = roberta_tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )
    
    roberta_enc = (
        np.array(roberta_enc_di["input_ids"]),
        np.array(roberta_enc_di["attention_mask"]),
        np.array(roberta_enc_di["token_type_ids"]),
    )
    return roberta_enc

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

def data_generator(train_df, val_df):

    X_train = regular_encode(train_df["post_message"].values, maxlen=MAX_LEN)
    # y_train = tf.keras.utils.to_categorical(train_df['Label'].values, num_classes=2)
    y_train = train_df["label"].values
    X_val = regular_encode(val_df["post_message"].values, maxlen=MAX_LEN)
    # y_val = tf.keras.utils.to_categorical(val_df['Label'].values, num_classes=2)
    y_val = val_df["label"].values

    train_dataset = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .repeat()
        .shuffle(1024)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )

    return train_dataset, valid_dataset

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def build_model(max_len=MAX_LEN):
    roberta_model = TFAutoModel.from_pretrained(roberta)

    roberta_input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="roberta_input_id"
    )
    roberta_attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="roberta_attention_mask"
    )
    roberta_token_type_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="roberta_token_type_ids"
    )

    roberta_sequence_output = roberta_model(
        roberta_input_word_ids,
        attention_mask=roberta_attention_mask,
        token_type_ids=roberta_token_type_ids,
        output_hidden_states=True,
        output_attentions=True,
    )[0]
    # n_hiddens = 4
    # print(len(roberta_sequence_output))

    # print(roberta_sequence_output[0].shape)
    # print(roberta_sequence_output[1].shape)

    # roberta_sequence_output = tf.concat([roberta_sequence_output[2][-i] for i in range(n_hiddens)] , axis=-1)


    roberta_sequence_output = roberta_sequence_output[:, 0, :]
    sequence_output = tf.concat(
        [ 
            roberta_sequence_output,

        ],
        axis=-1,
        name="sequence_output",
    )

    out = tf.keras.layers.Flatten()(sequence_output)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(out)

    model = tf.keras.models.Model(
        inputs=[
            roberta_input_word_ids,
            roberta_attention_mask,
            roberta_token_type_ids,  # roberta input
        ],
        outputs=out,
    )
    model.compile(
        tf.keras.optimizers.Adam(lr=5e-5),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC()],
    )

    return model

In [ ]:
%%time
model = build_model(max_len=MAX_LEN)
model.summary()

In [ ]:
exp = 'roberta-baseline' 
n_splits = 5
n_epochs = 5

DISPLAY=1 # USE display=1 FOR INTERACTIVE
kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
output_dir = f'../{exp}_models'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def scheduler(epoch):
    return 3e-5*0.2**epoch

In [ ]:
for fold, (idxT,idxV) in enumerate(kf.split(train_df)):
    print('*'*100)
    print(f'FOLD: {fold+1}/{n_splits}')
    
    K.clear_session()
    
    model = build_model(max_len=MAX_LEN)
        
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    model_dir = os.path.join(output_dir, f'Fold_{fold+1}.h5')

    sv = tf.keras.callbacks.ModelCheckpoint(model_dir, 
                                            monitor='val_auc', 
                                            verbose=1, 
                                            save_best_only=True,
                                            save_weights_only=True, 
                                            mode='auto', 
                                            save_freq='epoch')
    
    train_df_ = train_df.iloc[idxT]
    val_df_ = train_df.iloc[idxV]
    train_dataset, valid_dataset = data_generator(train_df_, val_df_)
    
    n_steps = train_df_.shape[0] // BATCH_SIZE
    train_history = model.fit(
        train_dataset,
        steps_per_epoch=n_steps,
        
        callbacks=[sv, 
            reduce_lr,
            # tb
            ],
        validation_data=valid_dataset,
        epochs=n_epochs
    )

In [ ]:
# model.predict(valid_dataset)

In [ ]:
X_test = regular_encode(test_df['post_message'].values, maxlen=MAX_LEN)
y_test = np.zeros((len(test_df),1))
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test,y_test))
    .batch(BATCH_SIZE)
)

In [ ]:
model = build_model(max_len=MAX_LEN)
preds = []
for i, file_name in enumerate(os.listdir(output_dir)):
    print('_'*80)
    
    K.clear_session()
    model_path = os.path.join(output_dir, file_name)
    
    print(f'Inferencing with model from: {model_path}')
    model.load_weights(model_path)

    pred = model.predict(test_dataset,
                         batch_size=128,
                         verbose=DISPLAY)
    # print(pred[])
    preds.append(pred)

In [ ]:
preds = np.mean(preds, axis=0)

In [ ]:
test_df["prediction"] = preds

In [ ]:
test_df["prediction"].to_csv("result.csv", header=False)